In [1]:
import pandas as pd
import pymysql

# DB 연결
conn = pymysql.connect(
    host="localhost", # MariaDB 서버 주소
    port=3306,
    user="webuser",       # 사용자명
    password="webuser",  # 비밀번호
    database="DDProject",  # 사용할 데이터베이스
    charset="utf8mb4"  # 한글 지원
)

cursor = conn.cursor()


# 테이블 크리에이트
create_table_sql = """CREATE TABLE IF NOT EXISTS crackdown (
    id INT AUTO_INCREMENT PRIMARY KEY,
    성별 VARCHAR(10),         -- 성별
    적발횟수 INT,    -- 적발횟수
    나이 INT,               -- 나이
    알코올농도 FLOAT,   -- 알코올농도
    측정거부 VARCHAR(10),       -- 측정거부
    측정일시 DATETIME, -- 측정일시
    관할경찰서 VARCHAR(100), -- 관할경찰서
    구 VARCHAR(20),
    발생년월 VARCHAR(50),
    적발시각 VARCHAR(10)
)"""
cursor.execute(create_table_sql)



# 1. 데이터 로드
df_crackdown_rate = pd.read_csv('../result/서울_단속_적발.csv', encoding='utf-8', header=0)
# 2. 데이터 변환 (성별 및 측정거부)
df_crackdown_rate['나이'] = df_crackdown_rate['나이'].replace('불명', None)
# 3. 날짜 변환 (문자열 → DATETIME)
df_crackdown_rate["측정일시"] = pd.to_datetime(df_crackdown_rate["측정일시"], errors="coerce")  # 변환 실패 시 NaT 처리

# 테이블 인서트
insert_sql = """
    INSERT INTO crackdown (성별, 적발횟수, 나이, 알코올농도, 측정거부, 측정일시, 관할경찰서, 구, 발생년월, 적발시각)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

data = df_crackdown_rate[["성별", "적발횟수", "나이", "알콜농도", "측정거부", "측정일시", "관할경찰서", "구", "발생년월","적발시각"]].itertuples(index=False, name=None)
cursor.executemany(insert_sql, data)

# 변경 사항 저장
conn.commit()

# 연결 종료
conn.close()
# null_rows = df_crackdown_rate[df_crackdown_rate.isnull().any(axis=1)]
# null_rows